# Goal
- Implement chained methods to perform ETL
- Make read_transform() do all the heavylifting of data transformation 

In [1]:
import pandas as pd
pd.read_csv("USA_Housing.csv", nrows=5)

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Address
0,79545.458574,5.682861,7.009188,4.09,23086.800503,1.059034e+06,"208 Michael Ferry Apt. 674\nLaurabury, NE 3701..."
1,79248.642455,6.002900,6.730821,3.09,40173.072174,1.505891e+06,"188 Johnson Views Suite 079\nLake Kathleen, CA..."
2,61287.067179,5.865890,8.512727,5.13,36882.159400,1.058988e+06,"9127 Elizabeth Stravenue\nDanieltown, WI 06482..."
3,63345.240046,7.188236,5.586729,3.26,34310.242831,1.260617e+06,USS Barnett\nFPO AP 44820
4,59982.197226,5.040555,7.839388,4.23,26354.109472,6.309435e+05,USNS Raymond\nFPO AE 09386


In [2]:
def lowcol(df):
    df.columns = df.columns.str.lower()
    return df


def replace_space(df):
    df.columns = df.columns.str.replace(" ", "_").str.replace(".", "")
    return df


def extract_state(df):
    df["state"] = df["address"].str.extract(", ([A-Z]{2})")
    return df


def read_transform(filename):
    df = (pd.read_csv(filename)
            .pipe(lowcol)
            .pipe(replace_space)
            .pipe(extract_state)
            .rename(columns={"price":"price_usd"})
          
        #creating additional columns
            .assign(price_jpy=lambda x: x["price_usd"].apply(round) * 110.5,)
          
        #dropping address column since now we have state column
            .drop("address", axis=1)
          
        #filtering the data 
            .query("avg_area_number_of_bedrooms >= 4"))
    return df
            

In [3]:
transformed = read_transform("USA_Housing.csv")
transformed.head()


,avg_area_income,avg_area_house_age,avg_area_number_of_rooms,avg_area_number_of_bedrooms,area_population,price_usd,state,price_jpy
0,79545.458574,5.682861,7.009188,4.09,23086.800503,1.059034e+06,NE,117023257.0
2,61287.067179,5.865890,8.512727,5.13,36882.159400,1.058988e+06,WI,117018174.0
4,59982.197226,5.040555,7.839388,4.23,26354.109472,6.309435e+05,NaN,69719201.5
5,80175.754159,4.988408,6.104512,4.04,26748.428425,1.068138e+06,KS,118029249.0
9,81885.927184,4.423672,8.167688,6.10,40149.965749,1.545155e+06,NaN,170739627.5
